#### Install dependencies

In [1]:
!uv add -q langchain-community

In [ ]:
!uv pip install -q pandas

In [2]:
!uv pip install -q https://github.com/zhenxiay/BasketIntelligence/releases/download/v1.0.0/basketintelligence-1.0.0-py3-none-any.whl

#### Test from langchain official Doc: https://python.langchain.com/docs/integrations/tools/sql_database/

In [ ]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

In [ ]:
import pandas as pd

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql_query(tables_query, engine).head(3)

name
0     Album
1    Artist
2  Customer

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
os.environ["no_proxy"] = "localhost, 127.0.0.1"
os.environ["NO_PROXY"] = "localhost, 127.0.0.1"

In [ ]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("openai:gpt-4o")

In [ ]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002112E664590>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002112E664590>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002112E664590>),
 QuerySQLCheckerTool(description='Use this tool to 

In [ ]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

In [ ]:
from langgraph.prebuilt import create_react_agent
from agent.utils.config import system_prompt

agent_executor = create_react_agent(llm, 
                                    toolkit.get_tools(), 
                                    prompt=system_prompt())

In [ ]:
example_query = "Which country's customers spent the most?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_LURPAUpyWw7SQOKDOw43cJa0)
 Call ID: call_LURPAUpyWw7SQOKDOw43cJa0
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_bl05ZUpsT8vCadwPHe2Q8mMv)
 Call ID: call_bl05ZUpsT8vCadwPHe2Q8mMv
  Args:
    table_names: Customer, Invoice
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company"

#### Test with own sqlite DB

In [ ]:
from BasketIntelligence.load_season_data import LoadSeasonData

loader = LoadSeasonData("2025","gcp-project-id","BasketIntelligence")

2025-09-24 14:38:21,386 - BasketIntelligence.utils.logger - INFO - Logger initialized
2025-09-24 14:38:21,389 - BasketIntelligence.utils.logger - INFO - Logger initialized


In [ ]:
 loader.load_data(
        data_source='per_game',
        db_type='sqlite',
        table_name='per_game',
        db_path='%USERPROFILE%/langgraph_dev_server/langgraph_dev_server/',
        db_name='basketball.sqlite'
    )

2025-09-24 14:40:18,926 - BasketIntelligence.utils.logger - INFO - Loading data for 'per_game' into 'sqlite'
2025-09-24 14:40:20,475 - BasketIntelligence.utils.logger - INFO - Dataframe created with 735 rows...
2025-09-24 14:40:20,556 - BasketIntelligence.utils.logger - INFO - per_game loaded to sqlite db C:/Users/YUZ1KA/langgraph_dev_server/langgraph_dev_server//basketball.sqlite successfully!
2025-09-24 14:40:20,559 - BasketIntelligence.utils.logger - INFO - Rows count: 735
2025-09-24 14:40:20,560 - BasketIntelligence.utils.logger - INFO - Data loaded for 'per_game' into 'sqlite' successfully!
2025-09-24 14:40:20,562 - BasketIntelligence.utils.logger - INFO - Rows count: 8


In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

connection = sqlite3.connect("basketball.sqlite")

engine = create_engine("sqlite:///basketball.sqlite", echo=False)

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql_query(tables_query, engine).head(3)

name
0  per_game

In [ ]:
tables_query = '''SELECT * FROM per_game 
                  WHERE Team = 'HOU' 
                  ORDER BY PTS DESC;'''
                  
pd.read_sql_query(tables_query, engine).head(3)

index   Rk          Player  Age Team Pos   G  GS    MP   FG  ...  ORB  DRB  \
0     50   43     Jalen Green   22  HOU  SG  82  82  32.9  7.4  ...  0.5  4.0   
1     63   56  Alperen Şengün   22  HOU   C  76  76  31.5  7.5  ...  3.4  6.9   
2    127  110   Amen Thompson   22  HOU  SF  69  42  32.2  5.6  ...  2.8  5.4   

    TRB  AST  STL  BLK  TOV   PF   PTS  Season  
0   4.6  3.4  0.9  0.3  2.5  1.5  21.0    2025  
1  10.3  4.9  1.1  0.8  2.6  2.8  19.1    2025  
2   8.2  3.8  1.4  1.3  2.0  2.4  14.1    2025  

[3 rows x 32 columns]

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase(engine)

In [ ]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("openai:gpt-4o")

In [ ]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
for tool in toolkit.get_tools():
    print(tool.name, ":", tool.description)

sql_db_query : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables : Input is an empty string, output is a comma-separated list of tables in the database.
sql_db_query_checker : Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


#### Result for the basketball intelligence DB case

In [ ]:
from langgraph.prebuilt import create_react_agent
from agent.utils.config import system_prompt

agent_executor = create_react_agent(llm, 
                                    toolkit.get_tools(), 
                                    prompt=system_prompt())

In [ ]:
example_query = "Which player has the most points per game from the Team HOU?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which player has the most points per game from the Team HOU?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_UTc55TsNHKipfNw3nmOfGmdw)
 Call ID: call_UTc55TsNHKipfNw3nmOfGmdw
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

per_game
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_QrisVgF1inWLB3rFfccrTDgs)
 Call ID: call_QrisVgF1inWLB3rFfccrTDgs
  Args:
    table_names: per_game
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE per_game (
	"index" INTEGER, 
	"Rk" INTEGER, 
	"Player" TEXT, 
	"Age" INTEGER, 
	"Team" TEXT, 
	"Pos" TEXT, 
	"G" INTEGER, 
	"GS" INTEGER, 
	"MP" REAL, 
	"FG" REAL, 
	"FGA" REAL, 
	"FG_pct" REAL, 
	"3P" REAL, 
	"3PA"

#### Test the Spark SQL Toolkit

In [1]:
from langchain_community.agent_toolkits import SparkSQLToolkit, create_spark_sql_agent
from langchain_community.utilities.spark_sql import SparkSQL

In [5]:
import os
print(os.environ.get('JAVA_HOME'))
print(os.environ.get('SPARK_HOME'))

C:\Program Files (x86)\Java\jre1.8.0_431
None


In [4]:
java_path = "C:\\Program Files (x86)\\Java\\jre1.8.0_431"
os.environ['JAVA_HOME'] = java_path

In [22]:
from pyspark import version

In [23]:
version.__version__

'4.0.1'

In [27]:
!java -version

java version "1.8.0_431"
Java(TM) SE Runtime Environment (build 1.8.0_431-b10)
Java HotSpot(TM) Client VM (build 25.431-b10, mixed mode)


In [9]:
import findspark
findspark.init()

In [27]:
findspark.find()

'c:\\Users\\zhenxiay\\langgraph_dev_server\\.venv\\Lib\\site-packages\\pyspark'

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
       .config("spark.driver.memory", "1g") \
       .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
schema = "langchain_example"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")

In [ ]:
from langgraph.prebuilt import create_react_agent
from src.agent.utils.config import system_prompt

agent_executor = create_react_agent(llm, 
                                    toolkit.get_tools(), 
                                    prompt=system_prompt())